In [167]:
#basic data analysis packages
import numpy as np
import pandas as pd

#basic data visualization packages
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [168]:
# load exp, methy, mirna data

"""
Method:
3 different datasets need to be created similarly (observation based on data)

Creating Title Row:
examination of the data showed that the title row values are either circled with "" or not, and separated by a space
names were created by selecting the first row of data, removing quotations (") and splitting by spaces, 
in addition, a category column was added so that the number of columns per row (474) matches with the number of titles (473)

Loading Data Without First Row (while setting "names" to title and delimiting by " ")
examination of single rows of data showed that rows begin with a category (some letters..some numbers)
row values are separated by single spaces
"""

dataframes = {'exp':'', 'methy':'', 'mirna':''}

def load_dataframe(filename):
    title_row = pd.read_csv('{}.csv'.format(filename), header=None, nrows=1)
    names = ['category'] + title_row.iloc[0, 0].replace('"', '').split(' ')
    dataframes[filename] = pd.read_csv('{}.csv'.format(filename), names=names, skiprows=1, sep=" ")
    
for x in dataframes.keys():
    load_dataframe(x)

In [169]:
# load survival data

"""
survival data needs to be loaded in a different way (based on data observation)
we will load it and add it to our dataframes dictionary (for consistency)
"""
survival = pd.read_csv('survival.csv', names=['PatientID', 'Survival', 'Death'], skiprows=1, sep='\t')

dataframes['survival'] = survival

In [174]:
# sample exp data
dataframes['exp'].head()

,category,TCGA.3N.A9WB.06,TCGA.3N.A9WC.06,TCGA.3N.A9WD.06,TCGA.BF.A1PU.01,TCGA.BF.A1PV.01,TCGA.BF.A1PX.01,TCGA.BF.A1PZ.01,TCGA.BF.A1Q0.01,TCGA.BF.A3DJ.01,...,TCGA.XV.AB01.06,TCGA.YD.A89C.06,TCGA.YD.A9TA.06,TCGA.YD.A9TB.06,TCGA.YG.AA3N.01,TCGA.YG.AA3O.06,TCGA.YG.AA3P.06,TCGA.Z2.A8RT.06,TCGA.Z2.AA3S.06,TCGA.Z2.AA3V.06
0,X..100130426,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.5522,0.0000,0.0000,0.0000,0.0000,0.0000
1,X..100133144,4.4709,5.2474,5.3404,0.3247,3.1889,0.5353,1.3536,22.9168,2.5883,...,0.0000,18.4309,42.3337,5.3220,0.5522,20.1673,0.6713,8.6977,3.2313,2.7530
2,X..100134869,6.4338,8.0016,3.8794,0.0000,3.3861,1.2026,3.8408,15.8059,5.6833,...,0.0000,67.8283,43.9516,13.8371,4.4174,7.8851,2.5253,2.5426,3.3926,2.4783
3,X..10357,143.5703,84.6612,118.8794,92.8766,163.9868,121.2026,161.8639,133.9102,180.9168,...,28.7741,257.5899,332.2118,122.2246,68.4705,75.8395,97.4853,103.9767,120.7776,29.0011
4,X..10431,1224.5557,560.4738,629.7872,544.8052,771.0699,793.5349,988.7709,988.4853,481.8197,...,388.4298,787.1695,512.6559,655.6679,691.8829,773.6375,709.6430,706.2016,366.0451,514.6314


In [164]:
# sample methy data
dataframes['methy'].head()

,category,TCGA.3N.A9WB.06,TCGA.3N.A9WC.06,TCGA.3N.A9WD.06,TCGA.BF.A1PU.01,TCGA.BF.A1PV.01,TCGA.BF.A1PX.01,TCGA.BF.A1PZ.01,TCGA.BF.A1Q0.01,TCGA.BF.A3DJ.01,...,TCGA.XV.AB01.01,TCGA.YD.A89C.06,TCGA.YD.A9TA.06,TCGA.YD.A9TB.06,TCGA.YG.AA3N.01,TCGA.YG.AA3O.06,TCGA.YG.AA3P.06,TCGA.Z2.A8RT.06,TCGA.Z2.AA3S.06,TCGA.Z2.AA3V.06
0,cg00001747,0.601759,0.662742,0.027184,0.017273,0.017128,0.477776,0.013000,0.617274,0.025718,...,0.893535,0.020740,0.019562,0.627104,0.157720,0.743431,0.958576,0.540532,0.033857,0.022691
1,cg00011616,0.307179,0.479190,0.734275,0.307967,0.109111,0.795075,0.092357,0.959433,0.889664,...,0.170478,0.152065,0.446951,0.257954,0.391654,0.445517,0.977751,0.157712,0.963803,0.593934
2,cg00013655,0.917332,0.591510,0.037064,0.415711,0.903536,0.291749,0.052929,0.638225,0.768412,...,0.955538,0.050278,0.139385,0.710827,0.771578,0.770109,0.042725,0.286802,0.792542,0.116445
3,cg00014152,0.782601,0.956630,0.642625,0.901424,0.974895,0.686262,0.143157,0.952074,0.972811,...,0.962496,0.955876,0.934961,0.971085,0.940844,0.974997,0.948403,0.652471,0.951512,0.962345
4,cg00028598,0.981708,0.934312,0.973639,0.981070,0.979136,0.962432,0.971122,0.967170,0.671760,...,0.419109,0.499426,0.596334,0.875208,0.766479,0.976866,0.984215,0.219373,0.695052,0.977684


In [165]:
# sample mirna data
dataframes['mirna'].head()

,category,TCGA.3N.A9WC.06,TCGA.3N.A9WD.06,TCGA.BF.A1PU.01,TCGA.BF.A1PV.01,TCGA.BF.A1PX.01,TCGA.BF.A1PZ.01,TCGA.BF.A1Q0.01,TCGA.BF.A3DJ.01,TCGA.BF.A3DL.01,...,TCGA.XV.AB01.06,TCGA.YD.A89C.06,TCGA.YD.A9TA.06,TCGA.YD.A9TB.06,TCGA.YG.AA3N.01,TCGA.YG.AA3O.06,TCGA.YG.AA3P.06,TCGA.Z2.A8RT.06,TCGA.Z2.AA3S.06,TCGA.Z2.AA3V.06
0,hsa.let.7a.1,14676.834734,25230.440999,23563.756602,14330.771178,13740.462752,16199.355403,10114.610880,16712.764049,35403.380723,...,10317.628480,4797.673951,9765.591431,7878.460725,13157.578121,12493.132639,23455.671756,10805.087024,12216.751081,24025.124723
1,hsa.let.7a.2,29175.042697,50426.377848,47180.136901,28590.374972,27700.049991,32326.559988,20268.091283,33860.575382,70851.588875,...,20559.441088,9682.578337,19545.557812,15938.581844,26378.793396,24907.572969,46879.348693,21513.398318,24460.759020,47953.696317
2,hsa.let.7a.3,14729.348617,25459.749831,23733.441179,14277.736854,13996.431180,16282.583317,10174.933610,17208.997960,35473.928808,...,10285.834727,4897.314490,9836.841186,8034.707489,13248.062199,12403.510843,23803.374490,10905.788400,12680.117674,24210.389886
3,hsa.let.7b,15377.019845,34387.698695,54133.369009,14988.630767,49402.682161,15234.151107,8981.281730,37588.292821,73681.595915,...,12514.954736,1542.629784,8806.219729,13989.262271,11230.764427,14622.196769,10265.711191,8431.948143,1860.659154,20967.711434
4,hsa.let.7c,306.330986,10490.699334,1727.530141,1659.272408,3080.541241,1126.121575,848.324435,255.635651,3177.603335,...,339.503915,96.762834,502.810772,2268.166510,2037.979847,528.987186,10879.780431,451.620072,1644.875692,1120.047059


In [166]:
# sample survival data
dataframes['survival'].head()

,PatientID,Survival,Death
0,TCGA-3N-A9WB-06,518.0,1.0
1,TCGA-3N-A9WC-06,2022.0,0.0
2,TCGA-3N-A9WD-06,395.0,1.0
3,TCGA-BF-A1PU-01,387.0,0.0
4,TCGA-BF-A1PV-01,14.0,0.0


In [175]:
# load clinical data data

"""
the data has a problem in line 38, 39:
they are series with 3 elements (total values: 105, unlike total 103 in other rows)
we discluded this data
"""

clinical_titles = pd.read_csv('melanoma.txt', header=None, nrows=1) 
clinical_data_titles = clinical_titles.iloc[0, 0].split('\t')

clinical_data = pd.read_csv('melanoma.txt', header=None, names=titles, skiprows=[0, 38,39], sep='\t') 

dataframes['clinical_data'] = clinical_data

In [176]:
#sample clinical data
dataframes['clinical_data'].head()

,sampleID,_EVENT,_INTEGRATION,_OS,_OS_IND,_OS_UNIT,_PATIENT,_RFS,_RFS_IND,_RFS_UNIT,...,_GENOMIC_ID_data/public/TCGA/SKCM/miRNA_HiSeq_gene,_GENOMIC_ID_TCGA_SKCM_RPPA,_GENOMIC_ID_TCGA_SKCM_mutation_bcm_gene,_GENOMIC_ID_TCGA_SKCM_mutation_broad_gene,_GENOMIC_ID_TCGA_SKCM_gistic2,_GENOMIC_ID_TCGA_SKCM_mutation,_GENOMIC_ID_TCGA_SKCM_exp_HiSeqV2_exon,_GENOMIC_ID_TCGA_SKCM_PDMRNAseqCNV,_GENOMIC_ID_TCGA_SKCM_PDMRNAseq,_GENOMIC_ID_TCGA_SKCM_exp_HiSeqV2_percentile
0,TCGA-3N-A9WB-06,1.0,TCGA-3N-A9WB-06,518.0,1.0,days,TCGA-3N-A9WB,NaN,NaN,days,...,NaN,NaN,NaN,TCGA-3N-A9WB-06A-11D-A38G-08,TCGA-3N-A9WB-06A-11D-A38F-01,NaN,4c243ea9-dfe1-42f0-a887-3c901fb38542,NaN,NaN,4c243ea9-dfe1-42f0-a887-3c901fb38542
1,TCGA-3N-A9WC-06,0.0,TCGA-3N-A9WC-06,2022.0,0.0,days,TCGA-3N-A9WC,2022.0,0.0,days,...,TCGA-3N-A9WC-06,F9B0B31B-24AB-4F30-8EF8-4A4ED282EF04,NaN,TCGA-3N-A9WC-06A-11D-A38G-08,TCGA-3N-A9WC-06A-11D-A38F-01,NaN,a64ae1f5-a189-4173-be13-903bd7637869,NaN,NaN,a64ae1f5-a189-4173-be13-903bd7637869
2,TCGA-3N-A9WD-06,1.0,TCGA-3N-A9WD-06,395.0,1.0,days,TCGA-3N-A9WD,NaN,NaN,days,...,TCGA-3N-A9WD-06,6AFC3678-9499-49A6-AD7A-30F3D185B4BD,NaN,TCGA-3N-A9WD-06A-11D-A38G-08,TCGA-3N-A9WD-06A-11D-A38F-01,NaN,ac19f7cf-670b-4dcc-a26b-db0f56377231,NaN,NaN,ac19f7cf-670b-4dcc-a26b-db0f56377231
3,TCGA-BF-A1PU-01,0.0,TCGA-BF-A1PU-01,387.0,0.0,days,TCGA-BF-A1PU,NaN,NaN,days,...,TCGA-BF-A1PU-01,7E74CAFD-C47B-4F98-A349-BBD2AC699D2D,TCGA-BF-A1PU-01A-11D-A19A-08,TCGA-BF-A1PU-01A-11D-A19A-08,NaN,TCGA-BF-A1PU-01A-11D-A19A-08,9ff4d588-da73-4e26-975d-2b098fe74984,NaN,NaN,9ff4d588-da73-4e26-975d-2b098fe74984
4,TCGA-BF-A1PV-01,0.0,TCGA-BF-A1PV-01,14.0,0.0,days,TCGA-BF-A1PV,14.0,0.0,days,...,TCGA-BF-A1PV-01,NaN,TCGA-BF-A1PV-01A-11D-A19A-08,TCGA-BF-A1PV-01A-11D-A19A-08,NaN,TCGA-BF-A1PV-01A-11D-A19A-08,a57841bb-fa68-4d84-85e0-2d5e4f3ce828,NaN,NaN,a57841bb-fa68-4d84-85e0-2d5e4f3ce828


In [129]:
#trying to make sense of the data
a = dataframes['survival'].merge(dataframes['clinical_data'], left_on='PatientID', right_on='sampleID', how='left')

a.head()

,PatientID,Survival,Death,sampleID,_EVENT,_INTEGRATION,_OS,_OS_IND,_OS_UNIT,_PATIENT,...,_GENOMIC_ID_data/public/TCGA/SKCM/miRNA_HiSeq_gene,_GENOMIC_ID_TCGA_SKCM_RPPA,_GENOMIC_ID_TCGA_SKCM_mutation_bcm_gene,_GENOMIC_ID_TCGA_SKCM_mutation_broad_gene,_GENOMIC_ID_TCGA_SKCM_gistic2,_GENOMIC_ID_TCGA_SKCM_mutation,_GENOMIC_ID_TCGA_SKCM_exp_HiSeqV2_exon,_GENOMIC_ID_TCGA_SKCM_PDMRNAseqCNV,_GENOMIC_ID_TCGA_SKCM_PDMRNAseq,_GENOMIC_ID_TCGA_SKCM_exp_HiSeqV2_percentile
0,TCGA-3N-A9WB-06,518.0,1.0,TCGA-3N-A9WB-06,1.0,TCGA-3N-A9WB-06,518.0,1.0,days,TCGA-3N-A9WB,...,NaN,NaN,NaN,TCGA-3N-A9WB-06A-11D-A38G-08,TCGA-3N-A9WB-06A-11D-A38F-01,NaN,4c243ea9-dfe1-42f0-a887-3c901fb38542,NaN,NaN,4c243ea9-dfe1-42f0-a887-3c901fb38542
1,TCGA-3N-A9WC-06,2022.0,0.0,TCGA-3N-A9WC-06,0.0,TCGA-3N-A9WC-06,2022.0,0.0,days,TCGA-3N-A9WC,...,TCGA-3N-A9WC-06,F9B0B31B-24AB-4F30-8EF8-4A4ED282EF04,NaN,TCGA-3N-A9WC-06A-11D-A38G-08,TCGA-3N-A9WC-06A-11D-A38F-01,NaN,a64ae1f5-a189-4173-be13-903bd7637869,NaN,NaN,a64ae1f5-a189-4173-be13-903bd7637869
2,TCGA-3N-A9WD-06,395.0,1.0,TCGA-3N-A9WD-06,1.0,TCGA-3N-A9WD-06,395.0,1.0,days,TCGA-3N-A9WD,...,TCGA-3N-A9WD-06,6AFC3678-9499-49A6-AD7A-30F3D185B4BD,NaN,TCGA-3N-A9WD-06A-11D-A38G-08,TCGA-3N-A9WD-06A-11D-A38F-01,NaN,ac19f7cf-670b-4dcc-a26b-db0f56377231,NaN,NaN,ac19f7cf-670b-4dcc-a26b-db0f56377231
3,TCGA-BF-A1PU-01,387.0,0.0,TCGA-BF-A1PU-01,0.0,TCGA-BF-A1PU-01,387.0,0.0,days,TCGA-BF-A1PU,...,TCGA-BF-A1PU-01,7E74CAFD-C47B-4F98-A349-BBD2AC699D2D,TCGA-BF-A1PU-01A-11D-A19A-08,TCGA-BF-A1PU-01A-11D-A19A-08,NaN,TCGA-BF-A1PU-01A-11D-A19A-08,9ff4d588-da73-4e26-975d-2b098fe74984,NaN,NaN,9ff4d588-da73-4e26-975d-2b098fe74984
4,TCGA-BF-A1PV-01,14.0,0.0,TCGA-BF-A1PV-01,0.0,TCGA-BF-A1PV-01,14.0,0.0,days,TCGA-BF-A1PV,...,TCGA-BF-A1PV-01,NaN,TCGA-BF-A1PV-01A-11D-A19A-08,TCGA-BF-A1PV-01A-11D-A19A-08,NaN,TCGA-BF-A1PV-01A-11D-A19A-08,a57841bb-fa68-4d84-85e0-2d5e4f3ce828,NaN,NaN,a57841bb-fa68-4d84-85e0-2d5e4f3ce828


In [133]:
# a.loc[a.sampleID.isnull()] #---> there are 2 rows with missing data --> need to deal with this...

#a.loc[a.Death != a._EVENT] #--> I think clinical_data._EVENT is the same as survival.Death

#a.loc[a.Survival != a._OS] #--> I think clinical_data._OS is the same as survival.Survival

In [144]:
# attempting to transpose data for simpler use --> might change data accordingly
n = dataframes['mirna']
transposed = n.T
transposed.columns = transposed.iloc[0]
transposed = transposed[1:]
transposed.head()

category,hsa.let.7a.1,hsa.let.7a.2,hsa.let.7a.3,hsa.let.7b,hsa.let.7c,hsa.let.7d,hsa.let.7e,hsa.let.7f.1,hsa.let.7f.2,hsa.let.7g,...,hsa.mir.941.3,hsa.mir.941.4,hsa.mir.942,hsa.mir.943,hsa.mir.944,hsa.mir.95,hsa.mir.96,hsa.mir.98,hsa.mir.99a,hsa.mir.99b
TCGA.3N.A9WC.06,14676.8,29175,14729.3,15377,306.331,1778.31,2772.89,43.7616,12410.4,1396.79,...,0,0,9.94581,0,1.1935,0.397832,0,138.446,56.0944,93333.1
TCGA.3N.A9WD.06,25230.4,50426.4,25459.7,34387.7,10490.7,1156.61,5706.84,49.5997,24356.3,1705.08,...,0,0,18.6897,0,4.31302,4.31302,6.46953,67.5706,1658.36,68524.5
TCGA.BF.A1PU.01,23563.8,47180.1,23733.4,54133.4,1727.53,693.774,6143.01,48.3279,18673.7,919.611,...,0,0,3.06844,0,0,0.767109,0.306844,67.8125,148.052,55153.5
TCGA.BF.A1PV.01,14330.8,28590.4,14277.7,14988.6,1659.27,474.384,5925.61,19.108,16942.3,751.645,...,0,0,8.18912,0.194979,0.974896,0.389958,0.194979,164.757,206.093,46455.5
TCGA.BF.A1PX.01,13740.5,27700,13996.4,49402.7,3080.54,684.134,2626,24.4334,7066.67,504.956,...,0,0,13.1863,0,12.4106,3.49048,0.387831,75.627,280.402,99058.6
